In [2]:
import numpy as np
import depth_calculation as dc

In [3]:
## Import data
rabi_file = 'D:\\Git\\DepthCalculations\\Rabi_old.asc'
xy8_file = 'D:\\Git\\DepthCalculations\\XY8-16_old.asc'

rabi_x, rabi_y = dc.get_data(rabi_file, data_columns=2)
xy8_x, xy8_y1, xy8_y2 = dc.get_data(xy8_file, data_columns=3)

In [16]:
## give values here
# order of XY8-N
N = 1

# gyromagnetic ratio electron
gamma_e = 2*np.pi*2.8025*1e4  # in MHz/T
gamma_e += 1e6  # to get Hz/T
# gyromagnetic ratio proton
gamma_p = 42.58 # in MHz/T
gamma_p *= 1e6  # to get Hz/T

# spin density
rho = 50       # in 1/nm^3
rho *= 1e27    # to get the spin density in 1/m^3

mu_0 = 1.256e-6
h = 6.62607e-34


In [17]:
h*gamma_p

2.821380606e-32

In [18]:
2*2.79*5.0507866*1e-27

2.8183389228000004e-26